In [1]:
import os
import diff_classifier.imagej as ij
import boto3
import os.path as op
import diff_classifier.aws as aws
import numpy as np

In [2]:
import cloudknot as ck
import diff_classifier.knotlets as kn

In [3]:
def download_and_track(filename):
    
    import diff_classifier.imagej as ij
    import diff_classifier.utils as ut
    import diff_classifier.aws as aws
    import os.path as op
    import pandas as pd
    
    aws.download_s3(filename, op.split(filename)[1])
    
    outfile = 'Traj_' + op.split(filename)[1].split('.')[0] + '.csv'
    local_im = op.join(os.getcwd(), op.split(filename)[1])
    if not op.isfile(outfile):
        ij.track(local_im, outfile, template=None, fiji_bin=None, radius=4.5, threshold=0., 
              do_median_filtering=True, quality=4.5, median_intensity=300.0, snr=0.0, 
              linking_max_distance=7.0, gap_closing_max_distance=10.0, max_frame_gap=2,
              track_displacement=10.0)

        aws.upload_s3(outfile, op.join(op.split(filename)[0], outfile))
    print("Done with tracking.  Should output file of name {}".format(op.join(op.split(filename)[0], outfile)))
    
    #return ut.csv_to_pd(outfile)

In [24]:
for track in to_track:
    download_and_track(track)

/home/ubuntu/Fiji.app/ImageJ-linux64 --ij2 --headless --run /tmp/tmpLT84R9.py


AttributeError: 'module' object has no attribute 'run'

In [23]:
to_track

['01_18_Experiment/P1/P1_S1_L_0001_0_0.tif',
 '01_18_Experiment/P1/P1_S1_L_0001_0_1.tif',
 '01_18_Experiment/P1/P1_S1_L_0001_0_2.tif',
 '01_18_Experiment/P1/P1_S1_L_0001_0_3.tif',
 '01_18_Experiment/P1/P1_S1_L_0001_1_0.tif',
 '01_18_Experiment/P1/P1_S1_L_0001_1_1.tif',
 '01_18_Experiment/P1/P1_S1_L_0001_1_2.tif',
 '01_18_Experiment/P1/P1_S1_L_0001_1_3.tif',
 '01_18_Experiment/P1/P1_S1_L_0001_2_0.tif',
 '01_18_Experiment/P1/P1_S1_L_0001_2_1.tif',
 '01_18_Experiment/P1/P1_S1_L_0001_2_2.tif',
 '01_18_Experiment/P1/P1_S1_L_0001_2_3.tif',
 '01_18_Experiment/P1/P1_S1_L_0001_3_0.tif',
 '01_18_Experiment/P1/P1_S1_L_0001_3_1.tif',
 '01_18_Experiment/P1/P1_S1_L_0001_3_2.tif',
 '01_18_Experiment/P1/P1_S1_L_0001_3_3.tif']

In [4]:
to_track = []
for i in range(0, 4):
    for j in range(0, 4):
        to_track.append('01_18_Experiment/P1/P1_S1_L_0001_{}_{}.tif'.format(i, j))

In [5]:
github_installs=('https://github.com/ccurtis7/diff_classifier.git@scaleup')
my_image = ck.DockerImage(func=download_and_track, base_image='arokem/python3-fiji:0.3', github_installs=github_installs)

#Step here: delete user info from newly created docker image.

In [6]:
docker_file = open(my_image.docker_path)
docker_string = docker_file.read()
docker_file.close()

In [7]:
cut1 = docker_string.find('# Create a default user')
cut2 = docker_string[cut1:].find('\n\n') + cut1
new_docker = docker_string[0:cut1]+docker_string[cut2:]
docker_overwrite = open(my_image.docker_path, 'w')
docker_overwrite.write(new_docker)
docker_overwrite.close()

In [8]:
knot = ck.Knot(name='download_and_track_with_my_image_12',
               docker_image = my_image,
               pars_policies=('AmazonS3FullAccess',))

In [9]:
result_futures = knot.map(to_track)

In [22]:
knot.view_jobs()

Job ID              Name                        Status   
---------------------------------------------------------
ad627159-ffd8-4f38-aca3-7af40df28640        download_and_track_with_my_image_12-15        FAILED   
00c86e54-ef69-498a-a759-2cafe64804da        download_and_track_with_my_image_12-8        SUCCEEDED
87f88e84-3a97-4b69-8782-e48e14513bdd        download_and_track_with_my_image_12-5        SUCCEEDED
02fcb3f9-9256-4429-a83e-145fa3dc3922        download_and_track_with_my_image_12-7        SUCCEEDED
ba422f40-124f-4149-a00d-de06dc3d81b7        download_and_track_with_my_image_12-10        SUCCEEDED
15508d9e-49df-4127-8d7a-a5b33915742c        download_and_track_with_my_image_12-6        SUCCEEDED
2e630317-e32b-4df9-886d-80bdbd9bd7e2        download_and_track_with_my_image_12-3        SUCCEEDED
0b855332-4e1c-44a1-9c98-a7e9e227f1c7        download_and_track_with_my_image_12-11        SUCCEEDED
97d6f033-8e37-46f1-9db1-a3c863362c51        download_and_track_with_my_image_12-14       

In [ ]:
knot.clobber()

In [ ]:
result_futures

In [ ]:
r0 = results[0]
r0.result()

In [ ]:
def download_and_split(filename):
    
    import diff_classifier.imagej as ij
    import diff_classifier.aws as aws
    import os.path as op
    
    local_name = op.split(filename)[1]
    DIR = op.split(filename)[0]
    try1 = filename.split('.')[0] + '_0_0.tif'
    try2 = filename.split('.')[0] + '_3_3.tif'
    
    s3 = boto3.client('s3')
    try:
        obj = s3.head_object(Bucket='ccurtis7.pup', Key=try1)
    except:
        try:
            obj = s3.head_object(Bucket='ccurtis7.pup', Key=try2)
        except:
            aws.download_s3(filename, local_name)
            names = ij.partition_im(local_name)
            for name in names:
                aws.upload_s3(name, op.join(op.split(filename)[0], name))
    print("Done with splitting.  Should output file of name {}".format(op.join(op.split(filename)[0], name)))

In [ ]:
pups = ["P1", "P2", "P3"]
slices = ["S1", "S2", "S3"]
hemis = ["R", "L"]
videos = 15
folder = '01_18_Experiment'

In [ ]:
missing = []
for i in range(10, 15):
    missing.append("P1_S2_R_00{}.tif".format(i))

for i in range(10, 15):
    missing.append("P2_S3_L_00{}.tif".format(i))
    
for i in range(0, 15):
    missing.append("P3_S3_L_{}.tif".format("%04d" % i))

In [ ]:
to_partition = []
for pup in pups:
    for slic in slices:
        for hemi in hemis:
            for i in range(0, videos):
                name = "{}/{}/{}_{}_{}_{}.tif".format(folder, pup, pup, slic, hemi, "%04d" % i)
                if not name in missing:
                    to_partition.append(name)

In [ ]:
for name in to_partition:
    download_and_split(name)

In [ ]:
!rm P1_S1_R_0000*

In [ ]:
to_partition

In [ ]:
import boto3

In [ ]:
s3 = boto3.resource('s3')

try:
    s3.Object("ccurtis7.pup", '01_18_Experiment/test_file.tif').load()

In [ ]:
s3 = boto3.client('s3')

result = s3.list_objects_v2(Bucket="ccurtis7.pup", Prefix="/01_18_Experiment/test_filebook.tif")

exists = False
if result:
    exists = True

print(exists)

In [ ]:
for obj in result.get('Contents', []):
    if obj['Key'] == "/01_18_Experiment/test_filebook.tif":
        print('Object here')
    else:
        print('Object not here')

In [ ]:
try:
    obj = s3.head_object(Bucket='ccurtis7.pup', Key="01_18_Experiment/test_file.tif")
except:
    print("Does not exist")